In [33]:
import json
import os

from openai import OpenAI

In [34]:
client = OpenAI(
    base_url="https://api.groq.com/openai/v1", api_key=os.getenv("GROQ_API_KEY")
)

In [35]:
def search_kb(question: str):
    """
    Load the whole knowledge base from the JSON file.
    (This is a mock function for demonstration purposes, we don't search)
    """
    with open("knowledge.json", "r") as f:
        return json.load(f)

In [36]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "search_kb",
            "description": "Get the answer to the user's question from the knowledge base.",
            "parameters": {
                "type": "object",
                "properties": {
                    "question": {"type": "string"},
                },
                "required": ["question"],
                "additionalProperties": False,
            },
            "strict": True,
        },
    }
]

In [37]:
system_prompt = "You are a helpful assistant that answers questions from the knowledge base about our e-commerce store."

messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": "What is the return policy?"},
]

In [38]:
completion = client.chat.completions.create(
    model="llama-3.1-8b-instant",
    messages=messages,
    tools=tools,
)

In [39]:
completion.model_dump()

{'id': 'chatcmpl-94b27411-a8d5-4628-bcaa-c3cbc11a28bb',
 'choices': [{'finish_reason': 'tool_calls',
   'index': 0,
   'logprobs': None,
   'message': {'content': None,
    'refusal': None,
    'role': 'assistant',
    'audio': None,
    'function_call': None,
    'tool_calls': [{'id': 'call_87gw',
      'function': {'arguments': '{"question": "What is the return policy?"}',
       'name': 'search_kb'},
      'type': 'function'}]}}],
 'created': 1739199565,
 'model': 'llama-3.1-8b-instant',
 'object': 'chat.completion',
 'service_tier': None,
 'system_fingerprint': 'fp_9cb648b966',
 'usage': {'completion_tokens': 18,
  'prompt_tokens': 253,
  'total_tokens': 271,
  'completion_tokens_details': None,
  'prompt_tokens_details': None,
  'queue_time': 0.02019535,
  'prompt_time': 0.017110407,
  'completion_time': 0.024,
  'total_time': 0.041110407},
 'x_groq': {'id': 'req_01jkr5ykfzfrtbd4mcd9hfhgha'}}

In [40]:
def call_function(name, args):
    if name == "search_kb":
        return search_kb(**args)


def checkforToolCalls(completion):
    if completion.choices[0].message.tool_calls:
        print("Tool call found")
    for tool_call in completion.choices[0].message.tool_calls:
        name = tool_call.function.name
        args = json.loads(tool_call.function.arguments)
        messages.append(completion.choices[0].message)

        result = call_function(name, args)
        messages.append(
            {
                "role": "tool",
                "tool_call_id": tool_call.id,
                "content": json.dumps(result),
            }
        )

In [41]:
checkforToolCalls(completion)

Tool call found


In [42]:
completion_2 = client.chat.completions.create(
    model="llama-3.1-8b-instant", messages=messages, tools=tools
)

In [43]:
final_response = completion_2.choices[0].message.content
final_response

'In order to meet our return policy, all returns need to be made within a 30-day purchasing window while including original receipts, and a refund should process within 5-7 days after.'

In [44]:
# No tools available

# messages2 = [
#     {"role": "system", "content": system_prompt},
#     {"role": "user", "content": "What is the weather in Tokyo?"},
# ]

In [45]:
# completion_3 = client.chat.completions.create(
#     model="llama-3.1-8b-instant",
#     messages=messages2,
#     tools=tools,
# )

# completion_3.choices[0].message.content

In [46]:
messages.append(completion_2.choices[0].message)

In [47]:
messages

[{'role': 'system',
  'content': 'You are a helpful assistant that answers questions from the knowledge base about our e-commerce store.'},
 {'role': 'user', 'content': 'What is the return policy?'},
 ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_87gw', function=Function(arguments='{"question": "What is the return policy?"}', name='search_kb'), type='function')]),
 {'role': 'tool',
  'tool_call_id': 'call_87gw',
  'content': '{"records": [{"id": 1, "question": "What is the return policy?", "answer": "Items can be returned within 30 days of purchase with original receipt. Refunds will be processed to the original payment method within 5-7 business days."}, {"id": 2, "question": "Do you ship internationally?", "answer": "Yes, we ship to over 50 countries worldwide. International shipping typically takes 7-14 business days and costs vary by destination. Please note that customs fees ma

In [48]:
messages.append({"role": "user", "content": "Do you accept UPI as payment method?"})

In [49]:
completion_3 = client.chat.completions.create(
    model="llama-3.1-8b-instant",
    messages=messages,
    tools=tools,
)

In [50]:
completion_3.model_dump()

{'id': 'chatcmpl-c3cdc0e5-135f-4eee-b28a-5f07edd802f4',
 'choices': [{'finish_reason': 'tool_calls',
   'index': 0,
   'logprobs': None,
   'message': {'content': None,
    'refusal': None,
    'role': 'assistant',
    'audio': None,
    'function_call': None,
    'tool_calls': [{'id': 'call_k7cc',
      'function': {'arguments': '{"question": "Do you accept UPI as payment method?"}',
       'name': 'search_kb'},
      'type': 'function'}]}}],
 'created': 1739199566,
 'model': 'llama-3.1-8b-instant',
 'object': 'chat.completion',
 'service_tier': None,
 'system_fingerprint': 'fp_9cb648b966',
 'usage': {'completion_tokens': 21,
  'prompt_tokens': 507,
  'total_tokens': 528,
  'completion_tokens_details': None,
  'prompt_tokens_details': None,
  'queue_time': 0.021240389999999998,
  'prompt_time': 0.033050429,
  'completion_time': 0.028,
  'total_time': 0.061050429},
 'x_groq': {'id': 'req_01jkr5ymm1ffb9k4n8e5gfdqsj'}}

In [51]:
checkforToolCalls(completion_3)

Tool call found


In [52]:
completion_4 = client.chat.completions.create(
    model="llama-3.1-8b-instant", messages=messages, tools=tools
)

In [53]:
messages

[{'role': 'system',
  'content': 'You are a helpful assistant that answers questions from the knowledge base about our e-commerce store.'},
 {'role': 'user', 'content': 'What is the return policy?'},
 ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_87gw', function=Function(arguments='{"question": "What is the return policy?"}', name='search_kb'), type='function')]),
 {'role': 'tool',
  'tool_call_id': 'call_87gw',
  'content': '{"records": [{"id": 1, "question": "What is the return policy?", "answer": "Items can be returned within 30 days of purchase with original receipt. Refunds will be processed to the original payment method within 5-7 business days."}, {"id": 2, "question": "Do you ship internationally?", "answer": "Yes, we ship to over 50 countries worldwide. International shipping typically takes 7-14 business days and costs vary by destination. Please note that customs fees ma

In [55]:
completion_4.choices[0].message.content

'No, we do not accept UPI as a payment method.'